In [1]:
import sys
import geopandas
import pandas as pd
import numpy as np
try:
    from osgeo import ogr, osr, gdal
except:
    sys.exit('ERROR: cannot find GDAL/OGR modules')

## load GIS vector data - waldorte

In [2]:
wo_geo = geopandas.read_file('/home/philipp/Data/edin_diss/GIS_test/joined.shp')

In [57]:
wo_geo.crs

{'init': 'epsg:31287'}

In [5]:
# filter abteilung -1
wo_geo = wo_geo.loc[wo_geo['ABTEILUNG'] != -1,:]

In [6]:
# create unique ID WO
wo_geo['WO'] = wo_geo['FORSTBETRI'].astype(str) + \
wo_geo['REVIER_NR'].astype(str).str.zfill(2) + \
wo_geo['ABTEILUNG'].astype(str).str.zfill(3) + \
wo_geo['UNTERABTEI'] + \
wo_geo['TEILFLAECH'].astype(str)

In [7]:
wo_geo['FLUGJAHR'] = wo_geo['FLUGJAHR'].fillna(0)
wo_geo['FLUGJAHR'] = wo_geo['FLUGJAHR'].astype(int)

In [8]:
wo_geo = wo_geo[wo_geo['FLUGJAHR'] != 0]

In [10]:
wo_geo.columns = ['obj_id', 'fb', 'fr', 'abt', 'uabt',
       'teilfl', 'color_code', 'link_id', 'id', 'admin', 'creation',
       'timeliness', 'length', 'area', 'year_fly', 'geometry', 'WO']

In [15]:
wo_geo = wo_geo.drop(['obj_id', 'color_code', 'link_id', 'id', 'creation', 'timeliness'], axis=1)

In [17]:
wo_geo.head()

,fb,fr,abt,uabt,teilfl,admin,length,area,year_fly,geometry,WO
4,171,1,506,3,1,529,2402.546849,11624.655500,2018,"POLYGON ((617139.640 493799.045, 617151.260 49...",1710150631
5,171,1,506,9,2,529,381.617564,3111.435375,2018,"POLYGON ((616697.800 493322.945, 616686.870 49...",1710150692
6,171,1,506,A,0,529,1367.653182,93101.347400,2018,"POLYGON ((617710.960 493854.385, 617742.710 49...",17101506A0
7,171,1,506,B,1,529,241.038183,611.014350,2018,"POLYGON ((617347.760 493783.145, 617356.300 49...",17101506B1
8,171,1,506,B,1,529,723.923359,11906.891550,2018,"POLYGON ((617454.430 493486.115, 617421.800 49...",17101506B1


## load GIS vector data - orthophoto metadata

In [34]:
meta_geo = geopandas.read_file('/home/philipp/Data/edin_diss/GIS_meta/flugjahr.shp')

In [10]:
meta_geo = meta_geo[['FLUGJAHR', 'geometry']]

In [35]:
meta_geo.head()

,FLUG_DAT,BOD_AUFL,SPEKTRUM,KAMERATYP,BEMERKUNG,MBL_NAME,DAT_LIEF_A,LFE_ID,LIEF_DAT,BL_NAME,FLUGJAHR,geometry
0,05.10.2018,0.2,RGBI,None,None,3621-47,23.05.2019,1619.0,17.04.2019,TIR,2018,"MULTIPOLYGON Z (((376255.203 299731.999 0.000,..."
1,08.08.2017,0.2,RGBI,None,None,4941-70,03.05.2018,1502.0,30.01.2018,OOE,2017,"MULTIPOLYGON Z (((528150.720 311746.428 0.000,..."
2,22.05.2016,0.2,RGBI,None,None,7827-24,19.06.2017,1413.0,22.12.2016,BGL,2016,"MULTIPOLYGON Z (((493732.849 274749.378 0.000,..."
3,26.08.2015,0.2,RGBI,None,None,1421-54,13.06.2016,1377.0,29.03.2016,TIR,2015,"MULTIPOLYGON Z (((224688.559 321076.871 0.000,..."
4,17.08.2013,0.0,RGBI,None,None,4318-06,27.05.2014,1126.0,02.04.2014,KTN,2013,"POLYGON Z ((401249.684 315727.538 0.000, 40000..."


### preform spatial join

In [ ]:
wo_geo = geopandas.sjoin(wo_geo, meta_geo, how='left', op='intersects')

In [ ]:
wo_geo.head()

## load SAP tax data

In [10]:
# get infromation about which Teiloperate were active in a giveb year

In [18]:
# set year
year = 2018

path_sap_info = '/home/philipp/Data/edin_diss/SAP_tax/edin_meta_data.xlsx'
sap_info = pd.read_excel(path_sap_info)

In [19]:
sap_info.head()

,FB,FR,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,171,1,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356
1,171,2,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356
2,171,3,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356
3,171,4,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356
4,171,5,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356


In [20]:
# array with all TOs in a given year
tos = sap_info.loc[:,year].unique()

In [21]:
tos

array([1030, 1048, 1049, 1053, 1052, 1100, 1050, 1051, 1083, 1042, 1073,
       1092, 1311, 1342, 1151, 1194, 1086, 1043, 1144, 1197, 1055, 1303,
       1085, 1345, 1302, 1140, 1312, 1065, 1066, 1067, 1088, 1044, 1196,
       1313, 1343, 1330, 1079, 1093, 1045, 1081, 1082, 1341, 1070, 1071,
       1156, 1250])

In [23]:
def get_data(tos):
    # set sap tax path directory
    path_sap_tax_dir = '/home/philipp/Data/edin_diss/SAP_tax'

    wo_sap_list = []

    for to in tos:
        # get fb
        fb = sap_info.loc[sap_info[year] == to, 'FB'].unique()[0]
        # create path to file
        path_sap_tax_file = path_sap_tax_dir + '/' + str(fb) + '/TO_' + str(to) + '.XLS'

        print(path_sap_tax_file)

        # read dat from file
        wo_sap_list.append(pd.read_csv(path_sap_tax_file, 
                                       sep='\t',
                                       encoding = "ISO-8859-1", 
                                       decimal=',', 
                                       error_bad_lines=False))
        # create unique ID WO
        #wo_sap['WO'] = wo_sap['Forstbetrieb'].astype(str) + \
        #wo_sap['Forstrevier'].astype(str) + \
        #wo_sap['Abteilung'].astype(str) + \
        #wo_sap['Unterabteil.'] + \
        #wo_sap['Teilfl.'].astype(str)

        #wo_sap_list.append(wo_sap)

    return pd.concat(wo_sap_list, ignore_index=True)

In [24]:
wo_sap = get_data(tos)

/home/philipp/Data/edin_diss/SAP_tax/171/TO_1030.XLS


/home/philipp/miniconda3/envs/geo/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (29,127,129,147,148,149) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


/home/philipp/Data/edin_diss/SAP_tax/171/TO_1048.XLS


/home/philipp/miniconda3/envs/geo/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (147,148,149) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


/home/philipp/Data/edin_diss/SAP_tax/171/TO_1049.XLS


...

/home/philipp/Data/edin_diss/SAP_tax/180/TO_1343.XLS
/home/philipp/Data/edin_diss/SAP_tax/180/TO_1330.XLS


/home/philipp/miniconda3/envs/geo/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (30,149,150) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


/home/philipp/Data/edin_diss/SAP_tax/180/TO_1079.XLS
/home/philipp/Data/edin_diss/SAP_tax/181/TO_1093.XLS
/home/philipp/Data/edin_diss/SAP_tax/181/TO_1045.XLS
/home/philipp/Data/edin_diss/SAP_tax/181/TO_1081.XLS
/home/philipp/Data/edin_diss/SAP_tax/181/TO_1082.XLS
/home/philipp/Data/edin_diss/SAP_tax/182/TO_1341.XLS
/home/philipp/Data/edin_diss/SAP_tax/182/TO_1070.XLS


/home/philipp/miniconda3/envs/geo/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (147,148) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


/home/philipp/Data/edin_diss/SAP_tax/182/TO_1071.XLS
/home/philipp/Data/edin_diss/SAP_tax/182/TO_1156.XLS
/home/philipp/Data/edin_diss/SAP_tax/182/TO_1250.XLS


In [25]:
wo_sap['Unterabteil.'].unique()

array(['B', 'C', 'A', 'D', 'G', 'F', 'E', 'L', 'H', 'N', 'I', 'K', '5',
       '3', '4', 'M', 'O', 'J', 'R', 'S', 'P', '9', '7', '8', '6', 'Z',
       'U', 'T', 'V', 'W', '1', 'Q', 'X', 'Y', '2'], dtype=object)

In [26]:
# create unique ID WO
wo_sap['WO'] = wo_sap['Forstbetrieb'].astype(str) + \
wo_sap['Forstrevier'].astype(str).str.zfill(2) + \
wo_sap['Abteilung'].astype(str).str.zfill(3) + \
wo_sap['Unterabteil.'] + \
wo_sap['Teilfl.'].astype(str)

In [27]:
wo_sap.head()

,Merkmalausprägung,AuswKatTyp,Teiloperats-ID,Forstbetrieb,Debitor,TO-Bezeichnung,Status,Beg. Laufzeit,Ende Laufzeit,Operat-ID,...,RÃ¼ckungsart,SchlÃ¤gerungsart,Repr. FlÃ¤che Schicht,GeschÃ¤ftsjahr,AbmaÃbeleg,MaÃnahme,GeschÃ¤ftsfeld,Repr. FlÃ¤che Baumart,FlÃ¤chenanteil,WO
0,0.0,NaN,1030,171,220442,1,2,01.01.2009,31.12.2019,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17101648B1
1,0.0,NaN,1030,171,220442,1,2,01.01.2009,31.12.2019,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17101648B1
2,0.0,NaN,1030,171,220442,1,2,01.01.2009,31.12.2019,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17101648B1
3,0.0,NaN,1030,171,220442,1,2,01.01.2009,31.12.2019,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17101648B1
4,0.0,NaN,1030,171,220442,1,2,01.01.2009,31.12.2019,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17101648B1


In [28]:
# stoe

# filter data
wo_sap_stoe = wo_sap.loc[wo_sap['Best.-Schicht'] == 0, ['WO', 'Forstbetrieb', 'Forstrevier', 'Abteilung', 
                                                        'Unterabteil.', 'Teilfl.', 'Beg. Laufzeit', 'Umtriebszeit', 
                                                        'Nebengrund Art', 'Ertragssituation', 'Bewirtschaftungsform', 
                                                        'Schutzwaldkategorie', 'Seehöhe', 'Exposition', 'Neigung', 
                                                        'Standorteinheit', 'Vegetationstyp', 'Wuchsgebiet']]

wo_sap_stoe.head()

,WO,Forstbetrieb,Forstrevier,Abteilung,Unterabteil.,Teilfl.,Beg. Laufzeit,Umtriebszeit,Nebengrund Art,Ertragssituation,Bewirtschaftungsform,Schutzwaldkategorie,Seehöhe,Exposition,Neigung,Standorteinheit,Vegetationstyp,Wuchsgebiet
0,17101648B1,171,1,648,B,1,01.01.2009,120,NaN,I,W,NaN,400.0,SO,27,87.0,BW,5.1
6,17101652C1,171,1,652,C,1,01.01.2009,120,NaN,I,W,NaN,400.0,SW,27,87.0,WW,5.1
12,17101736B4,171,1,736,B,4,01.01.2009,120,NaN,I,W,NaN,300.0,S,18,87.0,WW,5.1
21,17101658A1,171,1,658,A,1,01.01.2009,120,NaN,I,W,NaN,300.0,SW,18,87.0,WW,5.1
27,17101506D1,171,1,506,D,1,01.01.2009,120,NaN,I,W,NaN,300.0,NO,27,88.0,WW,5.1


In [29]:
# wood volume

# filter data
wo_sap_v = wo_sap.loc[wo_sap['Best.-Schicht.1'] > 0, ['WO', 'Vorrat / ha', 'Laubholzvorrat / ha', 
                                                      'Nadelholzvorrat / ha', 'Vorrat am Ort', 
                                                      'Laubholzvorrat Ort', 'Nadelholzvorrat Ort',]]

# group by WO (ID) and sum all values
wo_sap_v = wo_sap_v.groupby(['WO']).sum().reset_index()

wo_sap_v.head()

,WO,Vorrat / ha,Laubholzvorrat / ha,Nadelholzvorrat / ha,Vorrat am Ort,Laubholzvorrat Ort,Nadelholzvorrat Ort
0,17101506A0,331.2,229.68,101.52,3083.47,2138.32,945.15
1,17101506B1,236.7,208.18,28.52,295.88,260.23,35.65
2,17101506B2,13.0,13.00,0.00,7.28,7.28,0.00
3,17101506C1,175.0,175.00,0.00,428.75,428.75,0.00
4,17101506C2,89.0,72.90,16.10,263.44,215.78,47.66


In [30]:
# wood cuts

# filter data
wo_sap_ma = wo_sap.loc[wo_sap['Nutzungsnummer'] > 0, ['WO', 'Maßnahmenart', 'Massnahmengruppe', 'Angriffsfläche', 
                                                      'Nutzung LH', 'Nutzung NH', 'Nutzungssumme', 'Nutzdringlichkeit', 
                                                      'Bewpfl.', 'Zeitpunkt', 'Rückungsart', 'Schlägerungsart']]

wo_sap_ma.head()

,WO,Maßnahmenart,Massnahmengruppe,Angriffsfläche,Nutzung LH,Nutzung NH,Nutzungssumme,Nutzdringlichkeit,Bewpfl.,Zeitpunkt,Rückungsart,Schlägerungsart
5,17101648B1,RM,EN,4.0,1600.0,0.0,1600.0,2,3,2,30.0,1.0
10,17101652C1,DF,VN,3.0,100.0,0.0,100.0,2,1,2,30.0,1.0
11,17101652C1,ZE,EN,0.0,30.0,20.0,50.0,3,1,2,30.0,1.0
20,17101736B4,JP,WP,3.5,0.0,0.0,0.0,1,1,1,NaN,NaN
26,17101658A1,DF,VN,4.0,200.0,0.0,200.0,2,1,2,10.0,1.0


In [31]:
# filter just VN

In [32]:
wo_sap_maf = wo_sap_ma[wo_sap_ma['Massnahmengruppe'] == 'VN']
wo_sap_maf = wo_sap_maf[wo_sap_maf['Maßnahmenart'] != 'ZV']
wo_sap_maf = wo_sap_maf[wo_sap_maf['Maßnahmenart'] != 'UE']
wo_sap_maf = wo_sap_maf[wo_sap_maf['Maßnahmenart'] != 'LL']

In [33]:
wo_sap_maf['Maßnahmenart'].unique()

array(['DF', 'DE', 'ND', nan], dtype=object)

In [26]:
# get dublicated records
#wo_d = wo_sap_maf[wo_sap_maf.duplicated(subset = 'WO')]

In [34]:
# group by WO (ID) and sum all values
wo_sap_maf = wo_sap_maf.groupby(['WO']).sum().reset_index()

In [37]:
wo_sap_maf['ma'] = 'DF'

In [39]:
wo_sap_maf

,WO,Angriffsfläche,Nutzung LH,Nutzung NH,Nutzungssumme,Nutzdringlichkeit,Bewpfl.,Zeitpunkt,Rückungsart,Schlägerungsart,ma
0,17101506A0,9.3,350.0,150.0,500.0,2,1,2,30.0,1.0,DF
1,17101506C1,0.6,30.0,0.0,30.0,1,1,2,10.0,1.0,DF
2,17101506C2,3.2,165.0,0.0,165.0,3,2,4,45.0,5.0,DF
3,17101506D2,2.8,120.0,0.0,120.0,2,1,2,35.0,4.0,DF
4,17101506G1,1.9,60.0,30.0,90.0,1,1,2,35.0,4.0,DF
...,...,...,...,...,...,...,...,...,...,...,...
36851,18209366L1,2.5,0.0,120.0,120.0,1,1,1,35.0,4.0,DF
36852,18209366L2,0.1,0.0,10.0,10.0,2,1,1,35.0,4.0,DF
36853,18209366M2,1.9,0.0,80.0,80.0,1,1,1,35.0,4.0,DF
36854,18209367H2,0.6,0.0,20.0,20.0,2,1,1,30.0,2.0,DF


### merge all SAP data

In [40]:
# merge SAP stoe & SAP volume
wo_sap = pd.merge(wo_sap_stoe, wo_sap_v, how='left', on='WO', sort=False,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

# merge
wo_sap = pd.merge(wo_sap, wo_sap_maf, how='left', on='WO', sort=False,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

wo_sap.head()

,WO,Forstbetrieb,Forstrevier,Abteilung,Unterabteil.,Teilfl.,Beg. Laufzeit,Umtriebszeit,Nebengrund Art,Ertragssituation,...,Angriffsfläche,Nutzung LH,Nutzung NH,Nutzungssumme,Nutzdringlichkeit,Bewpfl.,Zeitpunkt,Rückungsart,Schlägerungsart,ma
0,17101648B1,171,1,648,B,1,01.01.2009,120,NaN,I,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17101652C1,171,1,652,C,1,01.01.2009,120,NaN,I,...,3.0,100.0,0.0,100.0,2.0,1.0,2.0,30.0,1.0,DF
2,17101736B4,171,1,736,B,4,01.01.2009,120,NaN,I,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17101658A1,171,1,658,A,1,01.01.2009,120,NaN,I,...,4.0,200.0,0.0,200.0,2.0,1.0,2.0,10.0,1.0,DF
4,17101506D1,171,1,506,D,1,01.01.2009,120,NaN,I,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
wo_sap.columns

Index(['WO', 'Forstbetrieb', 'Forstrevier', 'Abteilung', 'Unterabteil.',
       'Teilfl.', 'Beg. Laufzeit', 'Umtriebszeit', 'Nebengrund Art',
       'Ertragssituation', 'Bewirtschaftungsform', 'Schutzwaldkategorie',
       'Seehöhe', 'Exposition', 'Neigung', 'Standorteinheit', 'Vegetationstyp',
       'Wuchsgebiet', 'Vorrat / ha', 'Laubholzvorrat / ha',
       'Nadelholzvorrat / ha', 'Vorrat am Ort', 'Laubholzvorrat Ort',
       'Nadelholzvorrat Ort', 'Angriffsfläche', 'Nutzung LH', 'Nutzung NH',
       'Nutzungssumme', 'Nutzdringlichkeit', 'Bewpfl.', 'Zeitpunkt',
       'Rückungsart', 'Schlägerungsart', 'ma'],
      dtype='object')

In [42]:
wo_sap.columns = ['WO', 'fb_sap', 'fr_sap', 'abt_sap', 'uabt_sap',
       'teilfl_sap', 'start_term', 'uz', 'non_forest_type',
       'economy', 'ww_sw', 'sw_type',
       'sea_level', 'exp', 'slope', 'site_type', 'veg_type',
       'Wuchsgebiet', 'mass_ha', 'mass_ha_lh',
       'mass_ha_nh', 'mass_tot', 'mass_tot_lh',
       'mass_tot_nh', 'cut_area', 'cut_lh', 'cut_nh',
       'cut_sum', 'dr', 'bp', 'zp', 'ru', 'sg', 'ma']

In [45]:
wo_sap['year_fe'] = wo_sap['start_term'].str[-4:].astype(int)

In [46]:
wo_sap.to_csv('/home/philipp/Data/edin_diss/SAP_2018.csv')

In [9]:
wo_sap = pd.read_csv('/home/philipp/Data/edin_diss/SAP_2018.csv')

## load SAP nutz

In [10]:
def get_data_sap_nutz():
    # set sap tax path directory
    path_sap_nutz_dir = '/home/philipp/Data/edin_diss/SAP_nutz'

    sap_nutz_list = []

    for fb in range(171,183):
        # create path to file
        path_sap_file = path_sap_nutz_dir + '/FB' + str(fb) + '_2007_2019.xlsx'
        print(path_sap_file)
        # read dat from file
        sap_nutz_list.append(pd.read_excel(path_sap_file))

    return pd.concat(sap_nutz_list, ignore_index=True)

In [11]:
sap_nutz = get_data_sap_nutz()

/home/philipp/Data/edin_diss/SAP_nutz/FB171_2007_2019.xlsx
/home/philipp/Data/edin_diss/SAP_nutz/FB172_2007_2019.xlsx
/home/philipp/Data/edin_diss/SAP_nutz/FB173_2007_2019.xlsx
/home/philipp/Data/edin_diss/SAP_nutz/FB174_2007_2019.xlsx
/home/philipp/Data/edin_diss/SAP_nutz/FB175_2007_2019.xlsx
/home/philipp/Data/edin_diss/SAP_nutz/FB176_2007_2019.xlsx
/home/philipp/Data/edin_diss/SAP_nutz/FB177_2007_2019.xlsx
/home/philipp/Data/edin_diss/SAP_nutz/FB178_2007_2019.xlsx
/home/philipp/Data/edin_diss/SAP_nutz/FB179_2007_2019.xlsx
/home/philipp/Data/edin_diss/SAP_nutz/FB180_2007_2019.xlsx
/home/philipp/Data/edin_diss/SAP_nutz/FB181_2007_2019.xlsx
/home/philipp/Data/edin_diss/SAP_nutz/FB182_2007_2019.xlsx


In [12]:
# fill nan in 'Ergebnis'
sap_nutz = sap_nutz.fillna('Ergebnis')
# filter subtotals
sap_nutz = sap_nutz[sap_nutz['planmäßig'] != 'Ergebnis']
# reset index
sap_nutz = sap_nutz.reset_index(drop=True)

In [13]:
sap_nutz

,Abmaßjahr,Waldort,Nutzungsart,planmäßig,Menge Efm
0,2007,#/171/1/710C1,Räumung,planmäßig,175.18
1,2007,#/171/1/740A0,Räumung,planmäßig,314.12
2,2007,#/171/3/146C1,Durchforstung,planmäßig,28.40
3,2007,#/171/4/421B0,Schäden andere,zufällig,25.20
4,2007,#/171/5/294G0,Schäden Wind,zufällig,8.39
...,...,...,...,...,...
340306,2019,1355/182/5/297O5,Schäden Käfer,zufällig,1.74
340307,2019,1355/182/5/297O5,Schäden Wind,zufällig,5.22
340308,2019,1355/182/5/297O6,Schäden Käfer,zufällig,1.74
340309,2019,1355/182/5/297O7,Schäden Käfer,zufällig,1.16


In [14]:
## add Waldort
ids = sap_nutz['Waldort'].str.split('/')
# clear Waldort
sap_nutz['Waldort'] = ids.str[1] + ids.str[2].str.zfill(2) + ids.str[3].str.zfill(5)

In [15]:
sap_nutz['Waldort']

0         17101710C1
1         17101740A0
2         17103146C1
3         17104421B0
4         17105294G0
             ...    
340306    18205297O5
340307    18205297O5
340308    18205297O6
340309    18205297O7
340310    18205521G1
Name: Waldort, Length: 340311, dtype: object

In [16]:
# group by WO (ID) and sum all values
sap_nutz = sap_nutz.groupby(['Abmaßjahr','Waldort']).sum().reset_index()

In [17]:
# rename columns
sap_nutz.columns = ['cut_year', 'WO', 'cut_volume']
# change data type
sap_nutz['cut_year'] = sap_nutz['cut_year'].astype(int)
sap_nutz['cut_volume'] = sap_nutz['cut_volume'].astype(int)

In [18]:
sap_nutz

,cut_year,WO,cut_volume
0,2007,17101506C1,7.800
1,2007,17101507B1,4.550
2,2007,17101508D1,736.480
3,2007,17101510A2,8.450
4,2007,17101510C1,477.180
...,...,...,...
259961,2019,18209366H1,29.400
259962,2019,18209366H2,2.950
259963,2019,18209367F1,27.103
259964,2019,18209367K1,37.585


In [ ]:
###########################
# prepare data for merge
###########################

In [47]:
wo_sap

,WO,fb_sap,fr_sap,abt_sap,uabt_sap,teilfl_sap,start_term,uz,non_forest_type,economy,...,cut_lh,cut_nh,cut_sum,dr,bp,zp,ru,sg,ma,year_fe
0,17101648B1,171,1,648,B,1,01.01.2009,120,NaN,I,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
1,17101652C1,171,1,652,C,1,01.01.2009,120,NaN,I,...,100.0,0.0,100.0,2.0,1.0,2.0,30.0,1.0,DF,2009
2,17101736B4,171,1,736,B,4,01.01.2009,120,NaN,I,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
3,17101658A1,171,1,658,A,1,01.01.2009,120,NaN,I,...,200.0,0.0,200.0,2.0,1.0,2.0,10.0,1.0,DF,2009
4,17101506D1,171,1,506,D,1,01.01.2009,120,NaN,I,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226906,18208062B5,182,8,62,B,5,01.01.2016,140,NaN,I,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
226907,18208022A3,182,8,22,A,3,01.01.2016,140,NaN,I,...,0.0,150.0,150.0,1.0,1.0,2.0,10.0,1.0,DF,2016
226908,18208020B2,182,8,20,B,2,01.01.2016,140,NaN,I,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
226909,18209324C4,182,9,324,C,4,01.01.2016,140,NaN,I,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016


In [60]:
# merge GIS & SAP
wo = wo_geo.merge(wo_sap, on='WO')

In [64]:
wo.to_file('/home/philipp/Data/edin_diss/test_wo_2018.shp')

In [66]:
wo.crs

{'init': 'epsg:31287'}

In [61]:
wo.columns

Index(['fb', 'fr', 'abt', 'uabt', 'teilfl', 'admin', 'length', 'area',
       'year_fly', 'geometry', 'WO', 'fb_sap', 'fr_sap', 'abt_sap', 'uabt_sap',
       'teilfl_sap', 'start_term', 'uz', 'non_forest_type', 'economy', 'ww_sw',
       'sw_type', 'sea_level', 'exp', 'slope', 'site_type', 'veg_type',
       'Wuchsgebiet', 'mass_ha', 'mass_ha_lh', 'mass_ha_nh', 'mass_tot',
       'mass_tot_lh', 'mass_tot_nh', 'cut_area', 'cut_lh', 'cut_nh', 'cut_sum',
       'dr', 'bp', 'zp', 'ru', 'sg', 'ma', 'year_fe'],
      dtype='object')

In [63]:
wo[['year_fe', 'year_fly']]

,year_fe,year_fly
0,2009,2018
1,2009,2018
2,2009,2018
3,2009,2018
4,2009,2018
...,...,...
359732,2015,2013
359733,2015,2013
359734,2015,2013
359735,2015,2013


In [38]:
temp = wo.loc[wo['WO']=='17101506C1',['FE_year', 'FLUGJAHR']]

In [64]:
flug_year = int(temp.iloc[0,1])

In [65]:
flug_year

2018

In [101]:
wo_data = wo.copy()

In [102]:
wo_data

,OBJECTID,FORSTBETRI,REVIER_NR,ABTEILUNG,UNTERABTEI,TEILFLAECH,FARBCODE,LINKID,ID,VERWALTUNG,...,Angriffsfläche,Nutzung LH,Nutzung NH,Nutzungssumme,Nutzdringlichkeit,Bewpfl.,Zeitpunkt,Rückungsart,Schlägerungsart,FE_year
0,5.0,171,1,506,3,1,00000000003,{E77BB096-11C9-4939-9331-B4B305C4096A},1450402,529,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
1,6.0,171,1,506,9,2,00000000009,{99BC61C0-CE00-48BB-9140-A0C6D3EB5EA1},1450449,529,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
2,7.0,171,1,506,A,0,14000000000,{D5C7009D-5ED1-44F3-9A10-2EA9AC58094E},1450404,529,...,9.3,350.0,150.0,500.0,2.0,1.0,2.0,30.0,1.0,2009
3,8.0,171,1,506,B,1,28000400000,{E3993FFC-AC51-468D-9B23-A733AF0079C9},1450511,529,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
4,9.0,171,1,506,B,1,28000400000,{EA311B6D-6915-49CE-B17A-43499CA9077D},1450405,529,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370528,191316.0,177,2,150,K,3,11000000000,{8AFA0E7C-F3B3-4A29-9D07-0256BEF8009B},1618552,586,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015
370529,191317.0,177,2,150,K,4,11000000000,{E4CF5974-C830-4C9A-8314-AE15DA5A8F49},1618557,586,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015
370530,191318.0,177,2,150,M,0,48010030300,{C0791AFB-6FD1-419F-B5C9-B937A816768C},1618564,586,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015
370531,191319.0,177,2,150,M,0,48010030300,{AF31F1D8-4666-4DC0-958B-86E49406698F},1618574,586,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015


In [103]:
np_nutz = sap_nutz.values

In [140]:
np_wo = wo_data[['WO','FE_year', 'FLUGJAHR', 'SHAPE_Area', 'OBJECTID']].values

In [141]:
np_wo

array([['1710150631', 2009, 2018, 11624.6554996582, 5.0],
       ['1710150692', 2009, 2018, 3111.4353746856077, 6.0],
       ['17101506A0', 2009, 2018, 93101.34739962808, 7.0],
       ...,
       ['17702150M0', 2015, 2013, 8861.101586502582, 191318.0],
       ['17702150M0', 2015, 2013, 15139.0446723904, 191319.0],
       ['1770703574', 2017, 2013, 10201.78817499057, 203989.0]],
      dtype=object)

In [104]:
np_nutz

array([[2007, '17101506C1', 7.8],
       [2007, '17101507B1', 4.55],
       [2007, '17101508D1', 736.48],
       ...,
       [2019, '18209367F1', 27.103],
       [2019, '18209367K1', 37.585],
       [2019, '18209368A1', 224.048]], dtype=object)

In [144]:
## filtering and taking the sum of cuts 
## between year_fe and year_photo
## for every wo

%%time

tenth = np_wo.shape[0] // 20
state = np_wo.shape[0] // 20
state_per = 5

for i in range(np_wo.shape[0]):
    if i == state:
        print('{}%'.format(state_per))
        state_per += 5
        state += tenth
        
    # mask WO
    mask_wo = np_nutz[:,1] == np_wo[i, 0]
    # mask earlier than fe year
    mask_fe = np_nutz[:,0] >= np_wo[i, 1]
    # mask later than photo year
    mask_photo = np_nutz[:,0] < np_wo[i, 2]
    # create final mask
    mask = mask_wo * mask_fe * mask_photo
    # filter array with mask
    cut = np_nutz[mask,-1].sum()
    np_wo[i, -1] = cut

5%
10%
15%
20%
25%
30%
35%
40%
45%
50%
55%
60%
65%
70%
75%
80%
85%
90%
95%
100%
CPU times: user 2h 56s, sys: 5.71 s, total: 2h 1min 2s
Wall time: 2h 1min 13s


In [146]:
wo_cut = pd.DataFrame(np_wo, columns=['WO', 'year_fe', 'year_photo', 'area', 'cut'])

In [147]:
wo_cut

,WO,year_fe,year_photo,area,cut
0,1710150631,2009,2018,11624.7,0
1,1710150692,2009,2018,3111.44,0
2,17101506A0,2009,2018,93101.3,712.67
3,17101506B1,2009,2018,611.014,0
4,17101506B1,2009,2018,11906.9,0
...,...,...,...,...,...
370528,17702150K3,2015,2013,15996.2,0
370529,17702150K4,2015,2013,3216.48,0
370530,17702150M0,2015,2013,8861.1,0
370531,17702150M0,2015,2013,15139,0


In [148]:
wo_cut.to_csv('/home/philipp/Data/edin_diss/SAP_nutz/cut_till_2018.csv')

### load wo_cut

In [159]:
wo_cut = pd.read_csv('/home/philipp/Data/edin_diss/SAP_nutz/cut_till_2018.csv')

In [149]:
wo = wo.merge(wo_cut, on='WO')

In [155]:
wo['Vorrat am Ort'] = wo['Vorrat am Ort'].fillna(0)

In [171]:
wo.loc[(wo['Vorrat am Ort'] == 0) & (wo['cut'] != 0), ['WO', 'Nutzdringlichkeit', 'Vorrat am Ort', 'cut']]

,WO,Nutzdringlichkeit,Vorrat am Ort,cut
23,17101506D2,2.0,0.0,55.4
24,17101506D2,2.0,0.0,55.4
25,17101506D2,2.0,0.0,55.4
26,17101506D2,2.0,0.0,55.4
477,17101533E1,NaN,0.0,320.36
...,...,...,...,...
907637,18207532B2,NaN,0.0,62.9
907638,18207532B2,NaN,0.0,62.9
907639,18207532B2,NaN,0.0,62.9
907640,18207532B2,NaN,0.0,62.9


In [174]:
wo.columns = ['OBJECTID', 'fb', 'fr', 'abt', 'uabt',
       'TEILFLAECH', 'FARBCODE', 'LINKID', 'ID', 'VERWALTUNG', 'Erstellung',
       'Datenaktua', 'SHAPE_Leng', 'SHAPE_Area', 'FLUGJAHR', 'geometry', 'WO',
       'Unnamed', 'Forstbetrieb', 'Forstrevier', 'Abteilung',
       'Unterabteil', 'Teilfl', 'Beg_Laufzeit', 'Umtriebszeit',
       'Nebengrund_Art', 'Ertragssituation', 'Bewirtschaftungsform',
       'Schutzwaldkategorie', 'Seehoehe', 'Exposition', 'Neigung',
       'Standorteinheit', 'Vegetationstyp', 'Wuchsgebiet', 'Vorrat_ha',
       'Laubholzvorrat_ha', 'Nadelholzvorrat_ha', 'Vorrat_Ort',
       'Laubholzvorrat_Ort', 'Nadelholzvorrat_Ort', 'Angriffsflaeche',
       'Nutzung LH', 'Nutzung NH', 'Nutzungssumme', 'Nutzdringlichkeit',
       'Bewpfl', 'Zeitpunkt', 'Rueckungsart', 'Schlaegerungsart', 'FE_year',
       'year_fe', 'year_photo', 'area', 'cut']

In [175]:
wo.to_file('/home/philipp/Data/edin_diss/test_wo.shp')

In [158]:
wo['v_out'] = wo['cut'] / wo['Vorrat am Ort']

ZeroDivisionError: float division by zero

In [157]:
wo[['WO','Vorrat am Ort', 'cut']]

,WO,Vorrat am Ort,cut
0,1710150631,0.00,0
1,1710150692,0.00,0
2,17101506A0,3083.47,712.67
3,17101506B1,295.88,0
4,17101506B1,295.88,0
...,...,...,...
927706,17702150M0,1181.57,0
927707,17702150M0,1181.57,0
927708,17702150M0,1181.57,0
927709,17702150M0,1181.57,0


In [131]:
mask1 = np_nutz[:,1] == '17101506C1'

In [132]:
mask2 = np_nutz[:,0] >= 2009

In [133]:
mask3 = np_nutz[:,0] < 2018

In [134]:
mask = mask1*mask2*mask3

In [135]:
mask

array([False, False, False, ..., False, False, False])

In [139]:
np_nutz[mask,-1].sum()

24.05

In [87]:
wo_data

,OBJECTID,FORSTBETRI,REVIER_NR,ABTEILUNG,UNTERABTEI,TEILFLAECH,FARBCODE,LINKID,ID,VERWALTUNG,...,Angriffsfläche,Nutzung LH,Nutzung NH,Nutzungssumme,Nutzdringlichkeit,Bewpfl.,Zeitpunkt,Rückungsart,Schlägerungsart,FE_year
0,5.0,171,1,506,3,1,A,{E77BB096-11C9-4939-9331-B4B305C4096A},1450402,529,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
1,6.0,171,1,506,9,2,A,{99BC61C0-CE00-48BB-9140-A0C6D3EB5EA1},1450449,529,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
2,7.0,171,1,506,A,0,A,{D5C7009D-5ED1-44F3-9A10-2EA9AC58094E},1450404,529,...,9.3,350.0,150.0,500.0,2.0,1.0,2.0,30.0,1.0,2009
3,8.0,171,1,506,B,1,A,{E3993FFC-AC51-468D-9B23-A733AF0079C9},1450511,529,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
4,9.0,171,1,506,B,1,A,{EA311B6D-6915-49CE-B17A-43499CA9077D},1450405,529,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370528,191316.0,177,2,150,K,3,11000000000,{8AFA0E7C-F3B3-4A29-9D07-0256BEF8009B},1618552,586,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015
370529,191317.0,177,2,150,K,4,11000000000,{E4CF5974-C830-4C9A-8314-AE15DA5A8F49},1618557,586,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015
370530,191318.0,177,2,150,M,0,48010030300,{C0791AFB-6FD1-419F-B5C9-B937A816768C},1618564,586,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015
370531,191319.0,177,2,150,M,0,48010030300,{AF31F1D8-4666-4DC0-958B-86E49406698F},1618574,586,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015
